# 予実比較

In [2]:
import pandas as pd
from datetime import datetime, date, timedelta
from dateutil.relativedelta import relativedelta
import math

In [3]:
encoding = 'utf-8'

## 比較元プラン

In [23]:
plan = '01_first-plan'

### 基準日

In [5]:
# 月末日を指定すること
TargetDate = datetime(2021,10,30)

### プロジェクト期間

In [6]:
StartDate = datetime(2021,9 ,15)
EndDate   = datetime(2022,1,15)

### タスク別工程別実績

In [7]:
df_tasks_time = pd.read_csv('./data/task_time.csv', encoding=encoding)
df_tasks_time.fillna(0, inplace=True)
df_actual = df_tasks_time.query('実績!=0').copy()
df_actual['実績見込'] = df_actual.実績 + df_actual.残工数
df_actual['超過工数'] = df_actual.実績見込 - df_actual.工数
df_actual['予実対比'] = df_actual.実績見込 / df_actual.工数
df_actual

,No,ID,タイトル,分類,種別,優先度,順番,担当者,タスク名,工数,実績,残工数,実績見込,超過工数,予実対比
0,4,1125,Dを実装する,要件2,実装,1,1,田中,基本設計,30.0,40,0,40,10.0,1.333333
1,4,1125,Dを実装する,要件2,実装,1,1,田中,詳細設計,60.0,50,20,70,10.0,1.166667
3,1,1234,Aを調査する,残課題,調査,1,2,田中,基本設計,20.0,10,10,20,0.0,1.000000
6,2,1253,Bを調査する,残課題,調査,2,1,山田,概要作成,35.0,50,0,50,15.0,1.428571
7,2,1253,Bを調査する,残課題,調査,2,1,山田,調査,105.0,30,0,30,-75.0,0.285714


In [18]:
df_plan = df_tasks_time.query('実績==0').copy()
df_plan['予定工数'] = df_plan.工数
df_plan

,No,ID,タイトル,分類,種別,優先度,順番,担当者,タスク名,工数,実績,残工数,予定工数
2,4,1125,Dを実装する,要件2,実装,1,1,田中,実装,40.0,0,0,40.0
4,1,1234,Aを調査する,残課題,調査,1,2,田中,詳細設計,40.0,0,0,40.0
5,1,1234,Aを調査する,残課題,調査,1,2,田中,実装,60.0,0,0,60.0
8,2,1253,Bを調査する,残課題,調査,2,1,山田,資料作成,70.0,0,0,70.0
9,3,1324,Cを研究する,要件1,研究,5,1,山田,概要作成,105.0,0,0,105.0
10,3,1324,Cを研究する,要件1,研究,5,1,山田,調査,70.0,0,0,70.0
11,3,1324,Cを研究する,要件1,研究,5,1,山田,資料作成,140.0,0,0,140.0


In [47]:
df_actual_plan = pd.merge(df_actual.groupby('ID')[['実績','残工数']].sum().reset_index(), df_plan.groupby('ID')['予定工数'].sum().reset_index(), how='outer', on='ID')
df_actual_plan = df_actual_plan.fillna(0)
df_actual_plan.残工数 = df_actual_plan.残工数 + df_actual_plan.予定工数
df_actual_plan = df_actual_plan[['ID','実績','残工数']].fillna(0)
df_actual_plan['着地見込工数'] = df_actual_plan.実績 + df_actual_plan.残工数
df_actual_plan['進捗率'] = df_actual_plan.実績 / df_actual_plan.着地見込工数
df_actual_plan

,ID,実績,残工数,着地見込工数,進捗率
0,1125,90.0,60.0,150.0,0.600000
1,1234,10.0,110.0,120.0,0.083333
2,1253,80.0,70.0,150.0,0.533333
3,1324,0.0,315.0,315.0,0.000000


## シミュレート結果と比較

In [31]:
df_simulate_bym = pd.read_csv(f'../{plan}/out/simulate_tasks_bym.csv', encoding=encoding)
df_simulate_bym = df_simulate_bym.query(f"月 <= '{TargetDate}'", engine='python')
df_simulate_bym

,No,ID,タイトル,分類,種別,優先度,順番,担当者,工数,月,月工数
0,4,1125,Dを実装する,要件2,実装,1,1,田中,130.0,2021-09-15,42.0
1,2,1253,Bを調査する,残課題,調査,2,1,山田,210.0,2021-09-15,47.0
2,4,1125,Dを実装する,要件2,実装,1,1,田中,130.0,2021-10-01,84.0
3,2,1253,Bを調査する,残課題,調査,2,1,山田,210.0,2021-10-01,104.0


In [58]:
df_simulate_plan = df_simulate_bym.groupby('ID').agg({'月工数':'sum','工数':'max'}).reset_index()
df_simulate_plan = df_simulate_plan.rename(columns={'月工数':'予定消化工数','工数':'予定総工数'})
df_simulate_plan['予定進捗率'] = df_simulate_plan.予定消化工数 / df_simulate_plan.予定総工数
df_simulate_plan

,ID,予定消化工数,予定総工数,予定進捗率
0,1125,126.0,130.0,0.969231
1,1253,151.0,210.0,0.719048


In [62]:
df_task = pd.read_csv(f'../{plan}/data/tasks.csv', encoding=encoding)
columns = df_task.columns.tolist()
df_task = pd.merge(pd.merge(df_actual_plan, df_task, how='left', on='ID'), df_simulate_plan,  how='left', on='ID').fillna(0)
columns.extend(['予定消化工数', '実績', '予定進捗率', '進捗率', '予定総工数', '着地見込工数'])
df_task[columns]

,No,ID,タイトル,分類,種別,優先度,順番,担当者,予定消化工数,実績,予定進捗率,進捗率,予定総工数,着地見込工数
0,4,1125,Dを実装する,要件2,実装,1,1,田中,126.0,90.0,0.969231,0.600000,130.0,150.0
1,1,1234,Aを調査する,残課題,調査,1,2,田中,0.0,10.0,0.000000,0.083333,0.0,120.0
2,2,1253,Bを調査する,残課題,調査,2,1,山田,151.0,80.0,0.719048,0.533333,210.0,150.0
3,3,1324,Cを研究する,要件1,研究,5,1,山田,0.0,0.0,0.000000,0.000000,0.0,315.0
